In [ ]:
!pip install sklearn_crfsuite

     |████████████████████████████████| 747kB 4.8MB/s 


In [ ]:
from google.colab import drive
drive.mount('/content/drive')
from sklearn.metrics import f1_score

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
import os
from torch.utils import data
class SentenceGetter(data.Dataset):
    def __init__(self,type_data):
        super(SentenceGetter,self).__init__()
        path='/content/drive/My Drive/Colab/data/train'
        if(type_data=='test'):
          path='/content/drive/My Drive/Colab/data/test'
            
        arr=os.listdir(path)
        self.list_pos=[]
        self.list_sentence = []
        self.list_label = []
        self.sentences = []
        self.labels = []
        sentence = ""
        labels = ""
        pos=""
        for i in range(len(arr)):
            path_file = path + "/" + arr[i]

            file_data = open(path_file, encoding='utf-8')

            for line in file_data:

                if line == '\n':
                    sentence = sentence[:-1]
                    labels = labels[:-1]
                    pos=pos[:-1]
                    self.list_sentence.append(sentence)
                    self.list_label.append(labels)
                    self.list_pos.append(pos)
                    sentence = ""
                    labels = ""
                    pos=""

                else:
                    words = line.split("\t")
                    sentence = sentence + words[0] + " "
                    pos=pos+words[1]+" "
                    labels = labels + words[2].replace("\n", "") + " "
train=SentenceGetter('train')
list_sentences=train.list_sentence
print(len(list_sentences))
list_label=train.list_label
list_pos=train.list_pos
sentences=[[word for word in sentence.split(" ")] for sentence in list_sentences]
y_train=[[word for word in sentence.split(" ")] for sentence in list_label]
pos_train=[[word for word in sentence.split(" ")] for sentence in list_pos]


1418


In [ ]:
def is_number(s):
    try:
        complex(s) # for int, long, float and complex
    except ValueError:
        return False
 
    return True
def word2features(sent,pos,i):
  word=sent[i]
  features={
      'bias':1.0,
      'word':word,
      'word.is_number()': is_number(word),
      'word.isalpha()':word.isalpha(),
      'pos':pos[i]
  }
  if i > 1:
    word2=sent[i-2]
    features.update({
            '-2:word':word2,
            '-2:word.is_number()': is_number(word2),
            '-2:word.isalpha()':word2.isalpha(),
            '-2:pos':pos[i-2],
    })
  elif i> 0:
    word1=sent[i-1]
    features.update({
        '-1:word':word1,
        '-1:word.is_number()':is_number(word1),
        '-1:word.isalpha()':word1.isalpha(),
        '-1:pos':pos[i-1],
    })
  # else:
  #   features['BOS'] = True
  
  if i<len(sent)-2:
    word2=sent[i+2]
    features.update({
        '+2:word':word2,
        '+2:word.is_number()':is_number(word2),
        '+2:word.isalpha()':word2.isalpha(),
        '+2:pos':pos[i+2],
    })

  elif i<len(sent)-1:
    word1=sent[i+1]
    features.update({
        '+1:word':word1,
        '+1:word.is_number()':is_number(word1),
        '+1:word.isalpha()':word1.isalpha(),
        '+1:pos':pos[i+1],
    })
  # else:
  #   features['EOS']=True
  return features
def sent2features(sent,pos):
    return [word2features(sent,pos, i) for i in range(len(sent))]

In [ ]:
X_train = [sent2features(s,pos) for s,pos in zip(sentences,pos_train)]

In [ ]:
from sklearn_crfsuite import CRF

crf = CRF(algorithm='lbfgs',
          c1=0.1,
          c2=0.1,
          max_iterations=100,
          all_possible_transitions=False)

In [ ]:
crf.fit(X_train, y_train)

/usr/local/lib/python3.6/dist-packages/sklearn/base.py:197: FutureWarning: From version 0.24, get_params will raise an AttributeError if a parameter cannot be retrieved as an instance attribute. Previously it would return None.
  FutureWarning)


CRF(algorithm='lbfgs', all_possible_states=None, all_possible_transitions=False,
    averaging=None, c=None, c1=0.1, c2=0.1, calibration_candidates=None,
    calibration_eta=None, calibration_max_trials=None, calibration_rate=None,
    calibration_samples=None, delta=None, epsilon=None, error_sensitive=None,
    gamma=None, keep_tempfiles=None, linesearch=None, max_iterations=100,
    max_linesearch=None, min_freq=None, model_filename=None, num_memories=None,
    pa_type=None, period=None, trainer_cls=None, variance=None, verbose=False)

In [ ]:
from joblib import dump
dump(crf,'/content/drive/My Drive/Colab/model_crf/crf_model.joblib')

['/content/drive/My Drive/Colab/model_crf/crf_model.joblib']

In [ ]:
print(sentences[0])

['xe', 'máy', 'không', 'nhường', 'đường', 'cho', 'xe', 'trên', 'đường', 'ưu', 'tiên', 'thì', 'bị', 'phạt', 'bao', 'nhiêu', 'tiền']


In [ ]:
test=SentenceGetter('test')
list_sentences_test=test.list_sentence
list_label=test.list_label
list_pos=test.list_pos
test_sentences=[[word for word in sentence.split(" ")] for sentence in list_sentences_test]
pos_test=[[word for word in sentence.split(" ")] for sentence in list_pos]
y_test=[[word for word in sentence.split(" ")] for sentence in list_label]
X_test = [sent2features(s,pos) for s,pos in zip(test_sentences,pos_test)]


In [ ]:
y_predict=crf.predict(X_test)

In [ ]:
from sklearn_crfsuite import metrics
report = metrics.flat_classification_report(y_pred=y_predict, y_true=y_test)
print(report)

              precision    recall  f1-score   support

         B-a       0.94      0.88      0.91       281
        B-ac       1.00      1.00      1.00        10
       B-ano       1.00      0.56      0.71        18
        B-dl       1.00      0.96      0.98        26
       B-if1       1.00      0.84      0.91        38
       B-if2       1.00      0.50      0.67         6
       B-if3       0.79      0.75      0.77        64
       B-if4       0.67      0.69      0.68        32
         B-l       0.93      0.88      0.91        95
        B-qt       0.93      0.94      0.94       393
        B-sp       0.91      0.97      0.94        31
        B-ti       1.00      1.00      1.00        12
        B-tl       1.00      0.67      0.80         9
        B-tp       1.00      0.60      0.75         5
        B-tv       0.98      0.97      0.97       290
         B-v       0.97      1.00      0.98        32
         I-a       0.87      0.90      0.89       877
        I-ac       1.00    

In [ ]:
with open('/content/drive/My Drive/output.txt','w') as out:
  for k in range(len(test_sentences)):
    test_sentence=test_sentences[k]
    for i in range(len(test_sentence)):
      # print('{} {} {}'.format(test_sentence[i],y_test[k][i],y_predict[k][i]))
      out.write('{} {} {}\n'.format(test_sentence[i],y_test[k][i],y_predict[k][i]))
    # print()
    out.write('\n')


In [ ]:
del crf
# del train
# del test